# Processamento e Sumarização da Tabela de Pedidos

## Objetivo
Processar a tabela `tb_pedidos` mantendo specs históricas (no momento do pedido) para merge com TAREFCON.

## Diferenças vs ITENS
- **ITENS**: 1 linha por item (specs atuais)
- **PEDIDOS**: N linhas por item (histórico de pedidos com specs no momento)
- **Uso**: PEDIDOS é preferencial para merge (specs históricas)

## Estratégia
1. Filtrar pedidos relevantes (presentes na base de produção)
2. Remover pedidos cancelados/suspensos
3. Padronizar nomenclatura (match com ITENS)
4. Criar features derivadas (compatíveis com ITENS + específicas de pedido)
5. Validar e exportar

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

## 1. Carregamento e Análise Inicial

In [2]:
# Carregar dados
df_pedidos = pd.read_parquet("../../../data/raw/tb_pedidos.parquet")
df_merge_base = pd.read_parquet("df_final_base.parquet")

# Listas de itens e pedidos relevantes (presentes na produção)
list_itens = list(df_merge_base.CD_ITEM.unique())
list_pedidos = list(df_merge_base.CD_PEDIDO.unique())

print(f"Registros em tb_pedidos: {df_pedidos.shape[0]:,}")
print(f"Colunas em tb_pedidos: {df_pedidos.shape[1]}")
print(f"Itens únicos na base de produção: {len(list_itens):,}")
print(f"Pedidos únicos na base de produção: {len(list_pedidos):,}")

Registros em tb_pedidos: 406,890
Colunas em tb_pedidos: 87
Itens únicos na base de produção: 9,647
Pedidos únicos na base de produção: 230,174


## 2. Filtragem de Pedidos Relevantes

In [3]:
# Filtrar por itens e pedidos que existem na produção
df_pedidos_fil = df_pedidos[
    df_pedidos.CD_ITEM.isin(list_itens) & 
    df_pedidos.CD_PEDIDO.isin(list_pedidos)
].copy()

print(f"Após filtro por itens/pedidos produzidos: {df_pedidos_fil.shape[0]:,}")
print(f"Redução: {(1 - df_pedidos_fil.shape[0]/df_pedidos.shape[0])*100:.1f}%")

# Remover pedidos cancelados e suspensos
df_pedidos_fil = df_pedidos_fil[
    (df_pedidos_fil["ST_PEDIDO"] != "4") &  # Cancelados
    (df_pedidos_fil["FL_SUSPENSO"] == "0") &  # Suspensos
    (df_pedidos_fil["FL_SUSPOUCANCEL"] == "0")  # Suspensos ou cancelados
].reset_index(drop=True)

print(f"Após remover cancelados/suspensos: {df_pedidos_fil.shape[0]:,}")
print(f"Pedidos únicos: {df_pedidos_fil.CD_PEDIDO.nunique():,}")
print(f"Itens únicos: {df_pedidos_fil.CD_ITEM.nunique():,}")

Após filtro por itens/pedidos produzidos: 230,603
Redução: 43.3%
Após remover cancelados/suspensos: 230,543
Pedidos únicos: 230,076
Itens únicos: 7,623


## 3. Remoção de Colunas Irrelevantes

Remover colunas de:
- **Paletização**: logística pós-produção
- **Status processados**: já filtrados
- **Datas**: não relevantes para specs do produto
- **Referências**: redundantes

In [4]:
# Colunas a remover
cols_drop = [
    # Paletização (logística, não afeta produção)
    "CD_PALETE", "FL_PALETIZADO", "FL_AMARRADO",
    "QT_PECASPORPACOTE", "QT_PECASPORPALETE", 
    "QT_UNIDADESPORPALETE", "QT_PACOTESPORPALETE",
    "VL_ALTURAPACOTE", "VL_ALTURAPALETEFECHADO",
    "VL_COMPPALETEFECHADO", "VL_COMPPACOTE",
    "VL_LARGPACOTE", "VL_LARGPALETEFECHADO",
    "VL_PACOTESCOMPRIMENTO", "VL_PACOTESLARGURA", "VL_PACOTESALTURA",
    "VL_VOLUMEPACOTEFECHADOM3", "VL_VOLUMEPALETEFECHADOM3", 
    "VL_VOLUMEFECHADOPEDIDO",
    
    # Status (já processados nos filtros)
    "ST_PEDIDO", "FL_SUSPENSO", "FL_SUSPOUCANCEL",
    "TX_DESCRSTATUSPEDIDO",
    
    # Datas e tipo (remover por enquanto - pode adicionar depois se necessário)
    "DT_ENTREGA2", "DT_ENTREGAORIGINAL",
    "TX_DESCRTIPODOPEDIDO", "TX_DESCTIPOENTREGA", "FL_TIPOENTREGA",
    
    # Referências e metadados
    "CD_REFERENCIA",
]

# Verificar quais existem
cols_existentes = [col for col in cols_drop if col in df_pedidos_fil.columns]
df_pedidos_fil = df_pedidos_fil.drop(columns=cols_existentes)

print(f"Colunas removidas: {len(cols_existentes)}")
print(f"Colunas restantes: {df_pedidos_fil.shape[1]}")

Colunas removidas: 29
Colunas restantes: 58


## 4. Padronização de Nomenclatura

Renomear colunas para match com `nb_itens_processing.ipynb`:
- **FL_** = Flags binários
- **VL_** = Valores numéricos
- **QT_** = Quantidades
- **TX_** / **CAT_** = Categóricos
- **CD_** = Códigos identificadores

In [5]:
# Mapa de renomeação (match com ITENS)
rename_map = {
    # Testes de qualidade
    "FL_EXIGELAUDO": "FL_TESTE_EXIGELAUDO",
    "VL_GRAMATURA": "VL_TESTE_GRAMATURA",
    "QT_COBBINTMAXIMO": "VL_TESTE_COBB_INT_MAX",
    "VL_COLUNAMINIMO": "VL_TESTE_COLUNA_MINIMO",
    "VL_COMPRESSAO": "VL_TESTE_COMPRESSAO",
    
    # Flags e controles
    "CD_TIPOFT2": "FL_CONTROLE_ESPECIAL_IMPRESSAO",
    
    # LAP (serão convertidos para binário)
    "VL_LAPINTERNO": "FL_LAP_INTERNO",
    "VL_LAPNOCOMP": "FL_LAP_NO_COMPR",
    "QT_PROLONGLAP": "FL_PROLONG_LAP",
    
    # Categóricos
    "TX_COMPOSICAO": "CAT_COMPOSICAO",
    "CD_ESPELHO": "CAT_ESPELHO",
    "CD_FILME": "CAT_FILME",
}

df_pedidos_fil = df_pedidos_fil.rename(columns=rename_map)

print(f"Colunas renomeadas: {len(rename_map)}")
print("\nPrimeiras 15 colunas:")
print(list(sorted(df_pedidos_fil.columns))[:15])

Colunas renomeadas: 12

Primeiras 15 colunas:
['CAT_COMPOSICAO', 'CAT_ESPELHO', 'CAT_FILME', 'CD_FACA', 'CD_ITEM', 'CD_PEDIDO', 'FL_CHAPA', 'FL_CONTROLE_ESPECIAL_IMPRESSAO', 'FL_LAP_INTERNO', 'FL_LAP_NO_COMPR', 'FL_PROLONG_LAP', 'FL_REFILADO', 'FL_RESINAINTERNA', 'FL_TESTE_EXIGELAUDO', 'ID_CLIENTE']


## 5. Padronização de Valores

In [6]:
# 1. Converter LAP flags: -1 → 0 (padronizar como binário)
colunas_lap = ["FL_LAP_INTERNO", "FL_LAP_NO_COMPR"]
for col in colunas_lap:
    if col in df_pedidos_fil.columns:
        df_pedidos_fil[col] = df_pedidos_fil[col].replace(-1, 0)
        print(f"{col}: valores únicos = {sorted(df_pedidos_fil[col].unique())}")

# 2. Converter FL_PROLONG_LAP para binário
if "FL_PROLONG_LAP" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_PROLONG_LAP"] = (df_pedidos_fil["FL_PROLONG_LAP"] > 0).astype(int)
    print(f"FL_PROLONG_LAP: {df_pedidos_fil['FL_PROLONG_LAP'].value_counts().to_dict()}")

# 3. Padronizar FL_CONTROLE_ESPECIAL_IMPRESSAO: -1/2 → 0/1
if "FL_CONTROLE_ESPECIAL_IMPRESSAO" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_CONTROLE_ESPECIAL_IMPRESSAO"] = (
        df_pedidos_fil["FL_CONTROLE_ESPECIAL_IMPRESSAO"]
        .astype(str)
        .replace({"-1": "0", "2": "1"})
    )
    print(f"FL_CONTROLE_ESPECIAL_IMPRESSAO: {df_pedidos_fil['FL_CONTROLE_ESPECIAL_IMPRESSAO'].value_counts().to_dict()}")

# 4. Converter tipos numéricos
cols_int = ["FL_LAP_INTERNO", "FL_LAP_NO_COMPR", "FL_PROLONG_LAP", "QT_ARRANJO", "QT_NRCORES"]
for col in cols_int:
    if col in df_pedidos_fil.columns:
        df_pedidos_fil[col] = pd.to_numeric(df_pedidos_fil[col], errors="coerce").fillna(0).astype(int)

print("\n✅ Padronização de valores concluída")

FL_LAP_INTERNO: valores únicos = [np.float64(0.0), np.float64(1.0)]
FL_LAP_NO_COMPR: valores únicos = [np.float64(0.0), np.float64(1.0)]
FL_PROLONG_LAP: {0: 207770, 1: 22773}
FL_CONTROLE_ESPECIAL_IMPRESSAO: {'0': 230307, '1': 236}

✅ Padronização de valores concluída


## 6. Feature Engineering

### 6.1. Features de Produto (compatíveis com ITENS)

In [7]:
# 1. FL_TEM_LAP - Indicador de presença de LAP
df_pedidos_fil["FL_TEM_LAP"] = (
    (df_pedidos_fil.get("FL_LAP_INTERNO", 0) > 0) |
    (df_pedidos_fil.get("FL_LAP_NO_COMPR", 0) > 0) |
    (df_pedidos_fil.get("FL_PROLONG_LAP", 0) > 0)
).astype(int)

print(f"FL_TEM_LAP: {df_pedidos_fil['FL_TEM_LAP'].value_counts().to_dict()}")
print(f"  % com LAP: {df_pedidos_fil['FL_TEM_LAP'].mean()*100:.1f}%")

# 2. SCORE_QUALIDADE (0-3 para PEDIDOS - menos campos que ITENS)
df_pedidos_fil["SCORE_QUALIDADE"] = (
    df_pedidos_fil.get("FL_TESTE_EXIGELAUDO", 0).astype(int) +
    (df_pedidos_fil.get("VL_TESTE_COBB_INT_MAX", 0) > 0).astype(int) +
    (df_pedidos_fil.get("VL_TESTE_GRAMATURA", 0) > 0).astype(int)
)

print(f"\nSCORE_QUALIDADE: {df_pedidos_fil['SCORE_QUALIDADE'].value_counts().sort_index().to_dict()}")
print(f"  Média: {df_pedidos_fil['SCORE_QUALIDADE'].mean():.2f}")

# 3. TX_ONDA_TIPO - Extração do tipo de onda
if "CAT_COMPOSICAO" in df_pedidos_fil.columns:
    df_pedidos_fil["TX_ONDA_TIPO"] = (
        df_pedidos_fil["CAT_COMPOSICAO"]
        .astype(str)
        .str.extract(r'^([A-Z]+\d*)', expand=False)
    )
    print(f"\nTX_ONDA_TIPO - Top 10:")
    print(df_pedidos_fil["TX_ONDA_TIPO"].value_counts().head(10))

# 4. RAZAO_COMP_LARG - Razão dimensional
if "VL_COMPRIMENTO" in df_pedidos_fil.columns and "VL_LARGURA" in df_pedidos_fil.columns:
    df_pedidos_fil["RAZAO_COMP_LARG"] = (
        df_pedidos_fil["VL_COMPRIMENTO"] / df_pedidos_fil["VL_LARGURA"]
    ).replace([np.inf, -np.inf], np.nan)
    print(f"\nRAZAO_COMP_LARG:")
    print(f"  Média: {df_pedidos_fil['RAZAO_COMP_LARG'].mean():.2f}")
    print(f"  Min/Max: {df_pedidos_fil['RAZAO_COMP_LARG'].min():.2f} / {df_pedidos_fil['RAZAO_COMP_LARG'].max():.2f}")

# 5. FL_EXIG_PESOCAIXA
if "VL_PESOCAIXA" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_EXIG_PESOCAIXA"] = (df_pedidos_fil["VL_PESOCAIXA"] > 0).astype(int)
    print(f"\nFL_EXIG_PESOCAIXA: {df_pedidos_fil['FL_EXIG_PESOCAIXA'].value_counts().to_dict()}")

# 6. SCORE_COMPLEXIDADE (adaptado - sem vincos que não existem em PEDIDOS)
df_pedidos_fil["SCORE_COMPLEXIDADE"] = (
    df_pedidos_fil.get("QT_NRCORES", 0) +
    df_pedidos_fil["SCORE_QUALIDADE"] +
    df_pedidos_fil["FL_TEM_LAP"] * 2 +
    (df_pedidos_fil.get("FL_CONTROLE_ESPECIAL_IMPRESSAO", "0") == "1").astype(int) * 2
) / 6  # divisor ajustado

print(f"\nSCORE_COMPLEXIDADE:")
print(f"  Média: {df_pedidos_fil['SCORE_COMPLEXIDADE'].mean():.2f}")
print(f"  Min/Max: {df_pedidos_fil['SCORE_COMPLEXIDADE'].min():.2f} / {df_pedidos_fil['SCORE_COMPLEXIDADE'].max():.2f}")

FL_TEM_LAP: {0: 160826, 1: 69717}
  % com LAP: 30.2%

SCORE_QUALIDADE: {0: 3090, 1: 8374, 2: 9569, 3: 209510}
  Média: 2.85

TX_ONDA_TIPO - Top 10:
TX_ONDA_TIPO
A3     67242
A10    36599
A2     32780
A1     30459
A12    14990
A15    11830
A18     7167
B2      6664
B1      5937
A4      4621
Name: count, dtype: int64

RAZAO_COMP_LARG:
  Média: 2.58
  Min/Max: 0.27 / 6.30

FL_EXIG_PESOCAIXA: {1: 219697, 0: 10846}

SCORE_COMPLEXIDADE:
  Média: 0.82
  Min/Max: 0.00 / 1.67


### 6.2. Features Específicas de PEDIDOS

Features únicas relacionadas às características do pedido (não existem em ITENS).

In [8]:
# 1. VL_AMPLITUDE_QTD - Flexibilidade de quantidade
if all(col in df_pedidos_fil.columns for col in ["QT_PEDIDAMAX", "QT_PEDIDAMIN"]):
    df_pedidos_fil["VL_AMPLITUDE_QTD"] = (
        df_pedidos_fil["QT_PEDIDAMAX"] - df_pedidos_fil["QT_PEDIDAMIN"]
    )
    print(f"VL_AMPLITUDE_QTD:")
    print(f"  Média: {df_pedidos_fil['VL_AMPLITUDE_QTD'].mean():.0f} unidades")
    print(f"  Mediana: {df_pedidos_fil['VL_AMPLITUDE_QTD'].median():.0f} unidades")

# 2. VL_FLEXIBILIDADE_PCT - Amplitude como % da quantidade pedida
if "QT_PEDIDA" in df_pedidos_fil.columns and "VL_AMPLITUDE_QTD" in df_pedidos_fil.columns:
    df_pedidos_fil["VL_FLEXIBILIDADE_PCT"] = (
        (df_pedidos_fil["VL_AMPLITUDE_QTD"] / df_pedidos_fil["QT_PEDIDA"]) * 100
    ).replace([np.inf, -np.inf], np.nan)
    print(f"\nVL_FLEXIBILIDADE_PCT:")
    print(f"  Média: {df_pedidos_fil['VL_FLEXIBILIDADE_PCT'].mean():.1f}%")

# 3. FL_TEM_FLEXIBILIDADE - Flag binária
if "VL_AMPLITUDE_QTD" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_TEM_FLEXIBILIDADE"] = (
        df_pedidos_fil["VL_AMPLITUDE_QTD"] > 0
    ).astype(int)
    pct_flex = df_pedidos_fil["FL_TEM_FLEXIBILIDADE"].mean() * 100
    print(f"\nFL_TEM_FLEXIBILIDADE: {pct_flex:.1f}% dos pedidos têm flexibilidade")

# 4. TAMANHO_PEDIDO - Classificação por faixas
if "QT_PEDIDA" in df_pedidos_fil.columns:
    df_pedidos_fil["TAMANHO_PEDIDO"] = pd.cut(
        df_pedidos_fil["QT_PEDIDA"],
        bins=[0, 1000, 5000, 15000, np.inf],
        labels=["PEQUENO", "MEDIO", "GRANDE", "MUITO_GRANDE"]
    )
    print(f"\nTAMANHO_PEDIDO:")
    print(df_pedidos_fil["TAMANHO_PEDIDO"].value_counts())

# 5. QT_PEDIDOS_ITEM_COMPOSICAO - Popularidade da combinação item+composição
if all(col in df_pedidos_fil.columns for col in ["CD_ITEM", "CAT_COMPOSICAO", "CD_PEDIDO"]):
    df_pedidos_fil["QT_PEDIDOS_ITEM_COMPOSICAO"] = df_pedidos_fil.groupby(
        ["CD_ITEM", "CAT_COMPOSICAO"]
    )["CD_PEDIDO"].transform("nunique")
    print(f"\nQT_PEDIDOS_ITEM_COMPOSICAO:")
    print(f"  Média: {df_pedidos_fil['QT_PEDIDOS_ITEM_COMPOSICAO'].mean():.1f} pedidos")
    print(f"  Max: {df_pedidos_fil['QT_PEDIDOS_ITEM_COMPOSICAO'].max()} pedidos")

print("\n✅ Features específicas de PEDIDOS criadas")

VL_AMPLITUDE_QTD:
  Média: 1278 unidades
  Mediana: 800 unidades

VL_FLEXIBILIDADE_PCT:
  Média: 11.0%

FL_TEM_FLEXIBILIDADE: 100.0% dos pedidos têm flexibilidade

TAMANHO_PEDIDO:
TAMANHO_PEDIDO
GRANDE          123609
MEDIO            55726
MUITO_GRANDE     50536
PEQUENO            672
Name: count, dtype: int64

QT_PEDIDOS_ITEM_COMPOSICAO:
  Média: 231.7 pedidos
  Max: 2742 pedidos

✅ Features específicas de PEDIDOS criadas


## 7. Validação e Qualidade dos Dados

In [9]:
print("=" * 80)
print("VALIDAÇÃO DA QUALIDADE DOS DADOS")
print("=" * 80)

# 1. Resumo geral
print(f"\n1. RESUMO GERAL:")
print(f"  Total de registros: {len(df_pedidos_fil):,}")
print(f"  Total de colunas: {df_pedidos_fil.shape[1]}")
print(f"  Pedidos únicos: {df_pedidos_fil['CD_PEDIDO'].nunique():,}")
print(f"  Itens únicos: {df_pedidos_fil['CD_ITEM'].nunique():,}")

# 2. Valores nulos
print(f"\n2. VALORES NULOS:")
nulos = df_pedidos_fil.isnull().sum()
nulos_pct = (nulos / len(df_pedidos_fil) * 100).round(2)
colunas_com_nulos = nulos[nulos > 0].sort_values(ascending=False)

if len(colunas_com_nulos) > 0:
    print(f"  Colunas com nulos ({len(colunas_com_nulos)}):")
    for col in colunas_com_nulos.index[:10]:
        print(f"    {col}: {nulos[col]:,} ({nulos_pct[col]:.1f}%)")
else:
    print("  ✅ Nenhuma coluna com valores nulos")

# 3. Duplicatas
print(f"\n3. DUPLICATAS:")
duplicatas_completas = df_pedidos_fil.duplicated().sum()
print(f"  Linhas completamente duplicadas: {duplicatas_completas:,}")

# Duplicatas por chave composta
if "CD_PEDIDO" in df_pedidos_fil.columns and "CD_ITEM" in df_pedidos_fil.columns:
    duplicatas_chave = df_pedidos_fil.duplicated(subset=["CD_PEDIDO", "CD_ITEM"]).sum()
    print(f"  Duplicatas CD_PEDIDO + CD_ITEM: {duplicatas_chave:,}")

# 4. Estatísticas descritivas das features derivadas
print(f"\n4. FEATURES DERIVADAS:")
features_derivadas = [
    "FL_TEM_LAP", "SCORE_QUALIDADE", "SCORE_COMPLEXIDADE",
    "VL_AMPLITUDE_QTD", "FL_TEM_FLEXIBILIDADE", "QT_PEDIDOS_ITEM_COMPOSICAO"
]
features_existentes = [f for f in features_derivadas if f in df_pedidos_fil.columns]
print(df_pedidos_fil[features_existentes].describe().T)

print("\n" + "=" * 80)

VALIDAÇÃO DA QUALIDADE DOS DADOS

1. RESUMO GERAL:
  Total de registros: 230,543
  Total de colunas: 69
  Pedidos únicos: 230,076
  Itens únicos: 7,623

2. VALORES NULOS:
  ✅ Nenhuma coluna com valores nulos

3. DUPLICATAS:
  Linhas completamente duplicadas: 0
  Duplicatas CD_PEDIDO + CD_ITEM: 0

4. FEATURES DERIVADAS:
                               count         mean          std  min  \
FL_TEM_LAP                  230543.0     0.302403     0.459300  0.0   
SCORE_QUALIDADE             230543.0     2.845638     0.532541  0.0   
SCORE_COMPLEXIDADE          230543.0     0.821298     0.219616  0.0   
VL_AMPLITUDE_QTD            230543.0  1278.027669  1476.941071  0.0   
FL_TEM_FLEXIBILIDADE        230543.0     0.999510     0.022134  0.0   
QT_PEDIDOS_ITEM_COMPOSICAO  230543.0   231.737650   355.670276  1.0   

                                   25%         50%     75%           max  
FL_TEM_LAP                    0.000000    0.000000     1.0      1.000000  
SCORE_QUALIDADE               3

## 8. Exportação

In [10]:
import os

# Caminho de saída
output_path = "../../../data/ml/tb_pedidos_sumarizado.parquet"

# Garantir que diretório existe
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Salvar
df_pedidos_fil.to_parquet(output_path, index=False)

# Tamanho do arquivo
file_size_mb = os.path.getsize(output_path) / 1024**2

print("=" * 80)
print("EXPORTAÇÃO CONCLUÍDA")
print("=" * 80)
print(f"Arquivo: {output_path}")
print(f"Tamanho: {file_size_mb:.2f} MB")
print(f"Registros: {len(df_pedidos_fil):,}")
print(f"Colunas: {df_pedidos_fil.shape[1]}")
print("\n✅ Tabela pronta para merge com TAREFCON")

EXPORTAÇÃO CONCLUÍDA
Arquivo: ../../../data/ml/tb_pedidos_sumarizado.parquet
Tamanho: 11.20 MB
Registros: 230,543
Colunas: 69

✅ Tabela pronta para merge com TAREFCON


## 9. Resumo Final

In [11]:
print("=" * 80)
print("RESUMO DO PROCESSAMENTO")
print("=" * 80)

print(f"\nDados originais:")
print(f"  Registros: {df_pedidos.shape[0]:,}")
print(f"  Colunas: {df_pedidos.shape[1]}")

print(f"\nDados processados:")
print(f"  Registros: {df_pedidos_fil.shape[0]:,}")
print(f"  Colunas: {df_pedidos_fil.shape[1]}")

reducao_reg = (1 - df_pedidos_fil.shape[0]/df_pedidos.shape[0])*100
reducao_cols = (1 - df_pedidos_fil.shape[1]/df_pedidos.shape[1])*100

print(f"\nRedução:")
print(f"  Registros: {reducao_reg:.1f}%")
print(f"  Colunas: {reducao_cols:.1f}%")

print(f"\nFeatures criadas:")
features_novas = [
    "FL_TEM_LAP", "SCORE_QUALIDADE", "TX_ONDA_TIPO", "RAZAO_COMP_LARG",
    "FL_EXIG_PESOCAIXA", "SCORE_COMPLEXIDADE", "VL_AMPLITUDE_QTD",
    "VL_FLEXIBILIDADE_PCT", "FL_TEM_FLEXIBILIDADE", "TAMANHO_PEDIDO",
    "QT_PEDIDOS_ITEM_COMPOSICAO"
]
existentes = [f for f in features_novas if f in df_pedidos_fil.columns]
for feat in existentes:
    print(f"  ✅ {feat}")

print(f"\n{'=' * 80}")
print("PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
print("=" * 80)

RESUMO DO PROCESSAMENTO

Dados originais:
  Registros: 406,890
  Colunas: 87

Dados processados:
  Registros: 230,543
  Colunas: 69

Redução:
  Registros: 43.3%
  Colunas: 20.7%

Features criadas:
  ✅ FL_TEM_LAP
  ✅ SCORE_QUALIDADE
  ✅ TX_ONDA_TIPO
  ✅ RAZAO_COMP_LARG
  ✅ FL_EXIG_PESOCAIXA
  ✅ SCORE_COMPLEXIDADE
  ✅ VL_AMPLITUDE_QTD
  ✅ VL_FLEXIBILIDADE_PCT
  ✅ FL_TEM_FLEXIBILIDADE
  ✅ TAMANHO_PEDIDO
  ✅ QT_PEDIDOS_ITEM_COMPOSICAO

PROCESSAMENTO CONCLUÍDO COM SUCESSO!
